In [1]:
# Simon Dudtschak
# 20166103
#
# Based on: SOM-CNN by Ashraf Neisari
# Available: https://github.com/Ashsari/spam_review_detection

In [2]:
import pandas as pd
import numpy as np
import random
import time

from preprocessing import Preprocessing
from Clustering_words_SOM import Clustering
from Constructing_images_classification import Classify
import matplotlib.pyplot as plt

from SRD_minisom import MiniSom

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import collections
from itertools import chain

In [3]:
rand = random.randint(0,10000)
np.random.seed(rand)
random.seed(rand)

globalTime = time.perf_counter()

In [14]:
##############################################################
##                      PRE-PROCESSING                      ##
##############################################################
print("\nPRE-PROCESSING\n")

#importing the dataset
#@EXTENSION
dataset = pd.read_csv('final_mfrc_data.csv')
reviews = dataset.iloc[:,0]

#get labels and convert them to categorical format
list_words = []
for row in dataset["annotation"]:
    words = row.split(',')
    word_counts = collections.Counter(words)
    list_words.append(words)
    flat_list = list(chain.from_iterable(list_words))
unique_words = set(flat_list)


PRE-PROCESSING



In [5]:
#get labels and convert them to categorical format
y = dataset.iloc[:,3]
le = LabelEncoder()
y = le.fit_transform(y)
label = to_categorical(y, num_classes = 6)

In [6]:
#cleaning the vocabulary and tokenizing the reviews
print("Cleaning")
vocabulary, numWords, review_vocab, cleaned_reviews = Preprocessing.cleaning_reviews(reviews)

# keeping only unique words     
vocabulary = list(set(vocabulary))

# Here we provide the pretrained word embedding dictionary of glove with the vector dimension of 300
embedding_dim = 300
filepath ='glove.6B.300d.txt'

Cleaning


In [7]:
#calculate the TF-IDF of the vocabularies from the cleaned reviews 
df_TF_IDF, missed_tfidfW = Preprocessing.words_frequency(cleaned_reviews, vocabulary)

c:\users\simon\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
#Here we construct the embedding matrix of the fetched vocabulary from the reviews. 
#Each row of the matrix represents the corresponding word embedding vector
print("Embedding")
embedding_matrix = Preprocessing.create_embedding_matrix(filepath, vocabulary, embedding_dim )

#We fetch the list of the words that are not covered by the pretrained dictionary and later eliminate them from the vocabulary list  
nonzero_elements, not_foundWords, not_foundIndex = Preprocessing.not_found_words(embedding_matrix, vocabulary)


Embedding
The percentage of the covered words in the pretrained dictionary:  0.8740046287216099
number of not found words is:  3212


In [9]:
# Feature scaling make all features(the vectors values) between 0 and 1
print("Scaling")
X = embedding_matrix
sc = MinMaxScaler(feature_range = (0, 1))
X = sc.fit_transform(X)

Scaling


In [29]:
X.shape

(25493, 300)

In [45]:
dX = []

for i, item in enumerate(X):
    annotations = dataset['annotation'][i].split(',')
    tail = []
    for ann in unique_words:
        if ann in annotations: tail.append(1)
        else: tail.append(0)
    item = np.append(item,tail)
    dX.append(item)
        

In [50]:
dX = np.array(dX)
dX.shape
dX[1]

array([0.2603876 , 0.41899434, 0.23521784, 0.43829827, 0.34026442,
       0.40032493, 0.56003156, 0.70142562, 0.37277101, 0.51033442,
       0.32939404, 0.43210332, 0.45417201, 0.54493978, 0.36291196,
       0.45990825, 0.36147312, 0.4431532 , 0.68944668, 0.59450954,
       0.37213964, 0.41880861, 0.46740367, 0.48776777, 0.46945583,
       0.68560448, 0.68396719, 0.2540808 , 0.54489147, 0.42277035,
       0.51028944, 0.52877175, 0.53040765, 0.39871733, 0.61366329,
       0.38232154, 0.35523654, 0.30610896, 0.47129632, 0.5349207 ,
       0.38760022, 0.59530972, 0.61574073, 0.33836718, 0.68367886,
       0.41764642, 0.44392635, 0.53273837, 0.37731185, 0.42808208,
       0.39460295, 0.26339785, 0.76145547, 0.42879679, 0.47268829,
       0.41896417, 0.50591594, 0.31688774, 0.58912754, 0.44585269,
       0.52479806, 0.56856008, 0.78053674, 0.28766522, 0.38274871,
       0.64701814, 0.47042689, 0.4194883 , 0.62127616, 0.55399882,
       0.48691637, 0.46469735, 0.37194593, 0.33640692, 0.56686

In [51]:
##########################################################
##                      CLUSTERING                      ##
##########################################################
print("\nCLUSTERING\n")


#define the grid size for the SOM map by g.
#@EXTENSION
g = 50
X_of_map = g
y_of_map = g

#the input length passed to the SOM is the embedding vectors dimension 
input_len = X.shape[1]

#defining the neghborhood radius sigma as s and the learning rate
s = 1; l = 1 # the radius

#We create a SOM object and train it on the word vectors that were prepared in the pre-processing step
som = MiniSom(x = X_of_map, y= y_of_map, input_len = input_len, sigma = s, learning_rate = l, random_seed = 8)
#Randomly initialize the weights that are assigne to each grid cells
som.random_weights_init(X)

#trining the SOM
print("SOM Training")
#@EXTENSION
som.train_batch(data = X, num_iteration = 1000)



# #separating ham and spam reviews from in the dataset
# print("Seperate")
# ham , spam = Clustering.separate_ham_spam(reviews, y)
# #getting vocabulary list that are used in ham and spam reviews

# print("Cleaning")
# vocabulary_ham, numWords_ham, review_vocab_ham, cleaned_reviews_ham = Preprocessing.cleaning_reviews(ham)
# vocabulary_spam, numWords_spam, review_vocab_spam, cleaned_reviews_spam = Preprocessing.cleaning_reviews(spam)
# vocabulary_ham = list(set(vocabulary_ham))
# vocabulary_spam = list(set(vocabulary_spam))

# #creating a list of words that are common among the ham and spam reviews
# matches = list(set(vocabulary_spam) & set(vocabulary_ham))


CLUSTERING

SOM Training


In [52]:
#fetching the winning nodes of the word vectors from the SOM grid map
print("Calculate Win Map")
mappings = som.win_map(X)

Calculate Win Map


In [ ]:
##############################################################
##                      CLASSIFICATION                      ##
##############################################################
print("\nCLASSIFICATION\n")

#mappings = som.win_map(X)
map_cells = Classify.words_win_cel(mappings, vocabulary) 

#constructing the review images based on the feature word density
print("Create Density Image")
dense_img = Classify.create_dens_img(review_vocab, map_cells, vocabulary, not_foundWords, X_of_map, y_of_map)


CLASSIFICATION

Create Density Image


In [ ]:
#constructing the review images based on the feature word frequency 
print("Create Frequency Image")
frequency_img= Classify.create_freq_img(review_vocab, map_cells, vocabulary, missed_tfidfW, not_foundWords, X_of_map, y_of_map, df_TF_IDF)    



#reshaping the images to make them ready for the CNN
review_images = Classify.reshape_img(dense_img, frequency_img) 

#splitting the data to training and testing sets
X_train, X_test, y_train, y_test = train_test_split(review_images ,label, test_size=0.2,random_state= 252)
print("Create Model")
model = Classify.create_model(g, X_train, y_train, X_test, y_test)

In [ ]:
#getting the evaluation performance of the model
loss, tr_accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(tr_accuracy)) # 
loss, te_accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(te_accuracy)) # 
print('\n')

pred = (model.predict(X_test) > 0.5).astype("int32")
predict_classes = np.argmax(pred, axis=1)
expected_classes = np.argmax(y_test, axis = 1)
#print the classification report
evaluation = classification_report(expected_classes, predict_classes)
print(evaluation)

#plotting ROC value of the result
matrix = confusion_matrix(expected_classes, predict_classes)
ROC_value, fprx, tpry = Classify.ROC_Val(matrix)

globalTime = time.perf_counter() - globalTime
print("Total Time: ", globalTime)